In [1]:
import pyomo.environ as pyo
from pyomo.core.base.set import UnindexedComponent_set
from farmer_example_block_robby import build_block_model
from pyomo.dae.flatten import flatten_components_along_sets

Ipopt 3.13.3: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.3, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       21
                     variables with only lower bounds:       18
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequ

===Optimal solutions of two-stage stochastic problem with blocks===
Culture.         |  Wheat | Corn  | Sugar Beets |
Surface (acres)  |  0.0 | 200.0  | 300.0  |
First stage: s=1 (Above average)
Culture.         |  Wheat | Corn  | Sugar Beets |
Yield (T)        |  0.0 | 720.0 | 7200.0 |
Sales (T)        |  0.0 | 480.0 | 7200.0 |
Purchases (T)    |  1646.5 | 0.0   | -      |
First stage: s=2 (Average average)
Culture.         |  Wheat | Corn  | Sugar Beets |
Yield (T)        |  0.0 | 600.0 | 6000.0 |
Sales (T)        |  0.0 | 360.0   | 6000.0 |
Purchases (T)    |  1646.5   | 0.0   | -      |
First stage: s=3 (Below average)
Culture.         |  Wheat | Corn  | Sugar Beets |
Yield (T)        |  0.0 | 480.0 | 4800.0 |
Sales (T)        |  0.0 | 240.0   | 4800.0 |
Purchases (T)    |  1646.5   | 0.0   | -      |
Overall profit: $ 135600.0


In [3]:
def only_scenario_indexed():
    yields = [2.5, 3.0, 20.0]
    m = build_block_model(yields)

    sets = (m.scenarios,)
    # This partitions model components according to how they are indexed
    sets_list, vars_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Var,
    )
    print('sets_list:', sets_list)
    print('vars_list:', vars_list)
    assert len(sets_list) <= 2
    assert len(vars_list) <= 2

    for sets, vars in zip(sets_list, vars_list):
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            scalar_vars = vars
        elif len(sets) == 1 and sets[0] is m.scenarios:
            scenario_vars = vars
        else:
            # We only expect two cases here:
            # (a) unindexed
            # (b) indexed by scenario
            raise RuntimeError()

    sets_list, cons_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Constraint,
    )
    assert len(sets_list) <= 2
    assert len(sets_list) <= 2
    scenario_cons = []
    for sets, cons in zip(sets_list, cons_list):
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            scalar_cons = cons
        elif len(sets) == 1 and sets[0] is m.scenarios:
            scenario_cons = cons
        else:
            # We only expect two cases here:
            # (a) unindexed
            # (b) indexed by scenario
            raise RuntimeError()

    # The block hierarchy has been "flattened."
    # Not to be confused with "flattening" a high-dimension
    # index set into single-dimension index set.
    flattened_model = pyo.ConcreteModel()
    flattened_model.unindexed_vars = pyo.Reference(scalar_vars)
    for i, var in enumerate(scenario_vars):
        # var is already a reference.
        flattened_model.add_component("scenario_var_%s" % i, var)

    flattened_model.unindexed_constraints = pyo.Reference(scalar_cons)
    for i, con in enumerate(scenario_cons):
        flattened_model.add_component("scenario_con_%s" % i, con)

    flattened_model.obj = pyo.Reference(m.OBJ)

    solver = pyo.SolverFactory("ipopt")
    solver.solve(flattened_model, tee=True)


def all_sets():
    yields = [2.5, 3.0, 20.0]
    m = build_block_model(yields)

    # It makes sense to build the flattened model ahead of time
    # in this case, so we don't need to know what "set combinations"
    # we're looking for a priori
    flattened_model = pyo.ConcreteModel()

    sets = tuple(m.component_data_objects(pyo.Set))
    # This partitions model components according to how they are indexed
    sets_list, vars_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Var,
    )
    for i, (sets, vars) in enumerate(zip(sets_list, vars_list)):
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            flattened_model.unindexed_vars = pyo.Reference(vars)
        else:
            for j, var in enumerate(vars):
                flattened_model.add_component("var_%s_%s" % (i, j), var)

    sets_list, cons_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Constraint,
    )
    for i, (sets, cons) in enumerate(zip(sets_list, cons_list)):
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            flattened_model.unindexed_constraints = pyo.Reference(cons)
        else:
            for j, con in enumerate(cons):
                flattened_model.add_component("con_%s_%s" % (i, j), con)

    flattened_model.obj = pyo.Reference(m.OBJ)

    solver = pyo.SolverFactory("ipopt")
    solver.solve(flattened_model, tee=True)


def main():
    only_scenario_indexed()
    all_sets()


if __name__ == "__main__":
    main()

sets_list: [(<pyomo.core.base.global_set._UnindexedComponent_set object at 0x177c68580>,), (<pyomo.core.base.set.OrderedScalarSet object at 0x1793f2f20>,)]
vars_list: [[<pyomo.core.base.var._GeneralVarData object at 0x1049d8e20>, <pyomo.core.base.var._GeneralVarData object at 0x179409820>, <pyomo.core.base.var._GeneralVarData object at 0x1794098e0>], [<pyomo.core.base.var.IndexedVar object at 0x179369fd0>, <pyomo.core.base.var.IndexedVar object at 0x179369670>, <pyomo.core.base.var.IndexedVar object at 0x179369190>, <pyomo.core.base.var.IndexedVar object at 0x179369b50>, <pyomo.core.base.var.IndexedVar object at 0x179369970>, <pyomo.core.base.var.IndexedVar object at 0x1793699a0>]]
Ipopt 3.13.3: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/c

In [8]:
print(flattened_model.obj.value())

NameError: name 'flattened_model' is not defined

In [ ]:
def build_sp_model(yields):
    '''
    Code adapted from https://mpi-sppy.readthedocs.io/en/latest/examples.html#examples
    It specifies the extensive form of the two-stage stochastic programming
    
    Arguments:
        yields: Yield information as a list, following the rank [wheat, corn, beets]
        
    Return: 
        model: farmer problem model 
    '''
    model = ConcreteModel()
    
    all_crops = ["WHEAT", "CORN", "BEETS"]
    purchase_crops = ["WHEAT", "CORN"]
    sell_crops = ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"]
    scenarios = ["ABOVE","AVERAGE","BELOW"]
    
    # Fields allocation
    model.X = Var(all_crops, within=NonNegativeReals)
    # How many tons of crops to purchase in each scenario
    model.Y = Var(purchase_crops, scenarios, within=NonNegativeReals)
    # How many tons of crops to sell in each scenario
    model.W = Var(sell_crops, scenarios, within=NonNegativeReals)

    # Objective function
    model.PLANTING_COST = 150 * model.X["WHEAT"] + 230 * model.X["CORN"] + 260 * model.X["BEETS"]
    model.PURCHASE_COST_ABOVE = 238 * model.Y["WHEAT", "ABOVE"] + 210 * model.Y["CORN","ABOVE"]
    model.SALES_REVENUE_ABOVE = (
        170 * model.W["WHEAT", "ABOVE"] + 150 * model.W["CORN","ABOVE"]
        + 36 * model.W["BEETS_FAVORABLE","ABOVE"] + 10 * model.W["BEETS_UNFAVORABLE","ABOVE"])
    
    model.PURCHASE_COST_AVE = 238 * model.Y["WHEAT", "AVERAGE"] + 210 * model.Y["CORN","AVERAGE"]
    model.SALES_REVENUE_AVE = (
        170 * model.W["WHEAT", "AVERAGE"] + 150 * model.W["CORN","AVERAGE"]
        + 36 * model.W["BEETS_FAVORABLE","AVERAGE"] + 10 * model.W["BEETS_UNFAVORABLE","AVERAGE"])
    
    model.PURCHASE_COST_BELOW = 238 * model.Y["WHEAT", "BELOW"] + 210 * model.Y["CORN","BELOW"]
    model.SALES_REVENUE_BELOW = (
        170 * model.W["WHEAT", "BELOW"] + 150 * model.W["CORN","BELOW"]
        + 36 * model.W["BEETS_FAVORABLE","BELOW"] + 10 * model.W["BEETS_UNFAVORABLE","BELOW"])
    
    model.OBJ = Objective(
        expr=model.PLANTING_COST + 1/3*(model.PURCHASE_COST_ABOVE + model.PURCHASE_COST_AVE + model.PURCHASE_COST_BELOW)
        - 1/3*(model.SALES_REVENUE_ABOVE + model.SALES_REVENUE_AVE + model.SALES_REVENUE_BELOW),
        sense=minimize
    )

    # Constraints
    model.CONSTR= ConstraintList()

    model.CONSTR.add(summation(model.X) <= 500)
    model.CONSTR.add(yields[0] * model.X["WHEAT"] + model.Y["WHEAT","AVERAGE"] - model.W["WHEAT","AVERAGE"] >= 200)
    model.CONSTR.add(yields[0]*1.2 * model.X["WHEAT"] + model.Y["WHEAT","ABOVE"] - model.W["WHEAT","ABOVE"] >= 200)
    model.CONSTR.add(yields[0]*0.8 * model.X["WHEAT"] + model.Y["WHEAT","BELOW"] - model.W["WHEAT","BELOW"] >= 200)
    
    model.CONSTR.add(yields[1] * model.X["CORN"] + model.Y["CORN","AVERAGE"] - model.W["CORN","AVERAGE"] >= 240)
    model.CONSTR.add(yields[1]*1.2 * model.X["CORN"] + model.Y["CORN","ABOVE"] - model.W["CORN","ABOVE"] >= 240)
    model.CONSTR.add(yields[1]*0.8 * model.X["CORN"] + model.Y["CORN","BELOW"] - model.W["CORN","BELOW"] >= 240)
    
    model.CONSTR.add(
        yields[2] * model.X["BEETS"] - model.W["BEETS_FAVORABLE","AVERAGE"] - model.W["BEETS_UNFAVORABLE","AVERAGE"] >= 0
    )
    model.CONSTR.add(
        yields[2]*1.2 * model.X["BEETS"] - model.W["BEETS_FAVORABLE","ABOVE"] - model.W["BEETS_UNFAVORABLE","ABOVE"] >= 0
    )
    model.CONSTR.add(
        yields[2]*0.8 * model.X["BEETS"] - model.W["BEETS_FAVORABLE","BELOW"] - model.W["BEETS_UNFAVORABLE","BELOW"] >= 0
    )
    
    
    model.W["BEETS_FAVORABLE","AVERAGE"].setub(6000)
    model.W["BEETS_FAVORABLE","ABOVE"].setub(6000)
    model.W["BEETS_FAVORABLE","BELOW"].setub(6000)

    return model

In [ ]:
### calculate two-stage stochastic problem
yields_perfect = [2.5, 3, 20]
model = build_sp_model(yields_perfect)
solver = SolverFactory("ipopt")
solver.solve(model,tee=True)

profit_2stage = -value(model.OBJ)

print("===Optimal solutions of two-stage stochastic problem===")
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Surface (acres)  | ', f'{value(model.X["WHEAT"]):.1f}', '|', 
      f'{value(model.X["CORN"]):.1f}', ' |',
       f'{value(model.X["BEETS"]):.1f}',' |')
print('First stage: s=1 (Above average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]*1.2:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]*1.2:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]*1.2:.1f}','|')
print('Sales (T)        | ', f'{value(model.W["WHEAT","ABOVE"]):.1f}', '|', 
      f'{value(model.W["CORN","ABOVE"]):.1f}', '  |',
       f'{value(model.W["BEETS_FAVORABLE","ABOVE"]) + value(model.W["BEETS_UNFAVORABLE","ABOVE"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.Y["WHEAT","ABOVE"]):.1f}', '  |', 
      f'{value(model.Y["CORN","ABOVE"]):.1f}', '  |',
       '-','     |')

print('First stage: s=2 (Average average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]:.1f}','|')
print('Sales (T)        | ', f'{value(model.W["WHEAT","AVERAGE"]):.1f}', '|', 
      f'{value(model.W["CORN","AVERAGE"]):.1f}', '  |',
       f'{value(model.W["BEETS_FAVORABLE","AVERAGE"]) + value(model.W["BEETS_UNFAVORABLE","AVERAGE"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.Y["WHEAT","AVERAGE"]):.1f}', '  |', 
      f'{value(model.Y["CORN","AVERAGE"]):.1f}', '  |',
       '-','     |')

print('First stage: s=3 (Below average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]*0.8:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]*0.8:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]*0.8:.1f}','|')
print('Sales (T)        | ', f'{value(model.W["WHEAT","BELOW"]):.1f}', '|', 
      f'{value(model.W["CORN","BELOW"]):.1f}', '  |',
       f'{value(model.W["BEETS_FAVORABLE","BELOW"]) + value(model.W["BEETS_UNFAVORABLE","BELOW"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.Y["WHEAT","BELOW"]):.1f}', '  |', 
      f'{value(model.Y["CORN","BELOW"]):.1f}', '  |',
       '-','     |')
print('Overall profit: $',f"{profit_2stage:.1f}")

In [ ]:
def build_block_model(yields):
    '''
    Code adapted from https://mpi-sppy.readthedocs.io/en/latest/examples.html#examples
    
    Arguments:
        yields: Yield information as a list, following the rank [wheat, corn, beets]
        
    Return: 
        model: farmer problem model 
    '''
    model = ConcreteModel()
    
    # Define sets
    all_crops = ["WHEAT", "CORN", "BEETS"]
    purchase_crops = ["WHEAT", "CORN"]
    sell_crops = ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"]
    
    # define scenario 
    scenarios = ['ABOVE','AVERAGE','BELOW']
    
    # Crops field allocation
    model.X = Var(all_crops, within=NonNegativeReals)

    def construct_block(b, s):
        b.Y = Var(purchase_crops, within=NonNegativeReals)
        b.W = Var(sell_crops, within=NonNegativeReals)
    
    model.lsb = Block(scenarios, rule=construct_block)

    # Objective function
    model.PLANTING_COST = 150 * model.X["WHEAT"] + 230 * model.X["CORN"] + 260 * model.X["BEETS"]
    
    model.PURCHASE_COST = 238*sum(model.lsb[s].W["WHEAT"] for s in scenarios) + 210*sum(model.lsb[s].Y["CORN"] for s in scenarios)
    #model.SALES_REVENUE = (
    #    170*sum(model.lsb[s].W["WHEAT"] for s in scenarios) + 150*sum(model.lsb[s].W["CORN"] for s in scenarios)
    #    + 36*sum(model.lsb[s].W["BEETS_FAVORABLE"] for s in scenarios) + 10*sum(model.lsb[s].W["BEETS_UNFAVORABLE"] for s in scenarios)
    #)
    
    model.SALES_REVENUE_ABOVE = (
    170*model.lsb['ABOVE'].W["WHEAT"] + 150*model.lsb['ABOVE'].W["CORN"]
        + 36*model.lsb['ABOVE'].W["BEETS_FAVORABLE"] + 10*model.lsb['ABOVE'].W["BEETS_UNFAVORABLE"]
    )
    
    model.SALES_REVENUE_AVERAGE = (
    170*model.lsb['AVERAGE'].W["WHEAT"] + 150*model.lsb['AVERAGE'].W["CORN"]
        + 36*model.lsb['AVERAGE'].W["BEETS_FAVORABLE"] + 10*model.lsb['AVERAGE'].W["BEETS_UNFAVORABLE"]
    )
    
    model.SALES_REVENUE_BELOW = (
    170*model.lsb['BELOW'].W["WHEAT"] + 150*model.lsb['BELOW'].W["CORN"]
        + 36*model.lsb['BELOW'].W["BEETS_FAVORABLE"] + 10*model.lsb['BELOW'].W["BEETS_UNFAVORABLE"]
    )
    
    # Maximize the Obj is to minimize the negative of the Obj
    #model.OBJ = Objective(
    #    expr=model.PLANTING_COST + 1/3*model.PURCHASE_COST - 1/3*model.SALES_REVENUE,sense=minimize)
    
    model.OBJ = Objective(
        expr=model.PLANTING_COST + 1/3*model.PURCHASE_COST - 1/3*(model.SALES_REVENUE_ABOVE + 
                                                                  model.SALES_REVENUE_AVERAGE+
                                                                  model.SALES_REVENUE_BELOW), sense=minimize)


    # Constraints
    model.CONSTR= ConstraintList()

    model.CONSTR.add(summation(model.X) <= 500)
    model.CONSTR.add(yields[0] * model.X["WHEAT"] + model.lsb['AVERAGE'].Y["WHEAT"] - model.lsb['AVERAGE'].W["WHEAT"] >= 200)
    model.CONSTR.add(yields[0]*1.2 * model.X["WHEAT"] + model.lsb['ABOVE'].Y["WHEAT"] - model.lsb['ABOVE'].W["WHEAT"] >= 200)
    model.CONSTR.add(yields[0]*0.8 * model.X["WHEAT"] + model.lsb['BELOW'].Y["WHEAT"] - model.lsb['BELOW'].W["WHEAT"] >= 200)
    
    model.CONSTR.add(yields[1] * model.X["CORN"] + model.lsb['AVERAGE'].Y["CORN"] - model.lsb['AVERAGE'].W["CORN"] >= 240)
    model.CONSTR.add(yields[1]*1.2 * model.X["CORN"] + model.lsb['ABOVE'].Y["CORN"] - model.lsb['ABOVE'].W["CORN"] >= 240)
    model.CONSTR.add(yields[1]*0.8 * model.X["CORN"] + model.lsb['BELOW'].Y["CORN"] - model.lsb['BELOW'].W["CORN"] >= 240)
    
    
    model.CONSTR.add(yields[2] * model.X["BEETS"] - model.lsb['AVERAGE'].W["BEETS_FAVORABLE"] - model.lsb['AVERAGE'].W["BEETS_UNFAVORABLE"] >= 0)
    model.CONSTR.add(yields[2]*1.2 * model.X["BEETS"] - model.lsb['ABOVE'].W["BEETS_FAVORABLE"] - model.lsb['ABOVE'].W["BEETS_UNFAVORABLE"] >= 0)
    model.CONSTR.add(yields[2]*0.8 * model.X["BEETS"] - model.lsb['BELOW'].W["BEETS_FAVORABLE"] - model.lsb['BELOW'].W["BEETS_UNFAVORABLE"] >= 0)
    
    
    #model.CONSTR.add(model.lsb["AVERAGE"].W["BEETS_FAVORABLE"] <= 6000)
    #model.CONSTR.add(model.lsb["ABOVE"].W["BEETS_FAVORABLE"] <= 6000)
    #model.CONSTR.add(model.lsb["BELOW"].W["BEETS_FAVORABLE"] <= 6000)
    
    model.lsb['AVERAGE'].W["BEETS_FAVORABLE"].setub(6000)
    model.lsb['ABOVE'].W["BEETS_FAVORABLE"].setub(6000)
    model.lsb['BELOW'].W["BEETS_FAVORABLE"].setub(6000)

    return model

In [ ]:
### calculate two-stage stochastic problem
yields_perfect = [2.5, 3, 20]
model = build_block_model(yields_perfect)
solver = SolverFactory("ipopt")
solver.solve(model,tee=True)

profit_2stage = -value(model.OBJ)


print("===Optimal solutions of two-stage stochastic problem===")
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Surface (acres)  | ', f'{value(model.X["WHEAT"]):.1f}', '|', 
      f'{value(model.X["CORN"]):.1f}', ' |',
       f'{value(model.X["BEETS"]):.1f}',' |')
print('First stage: s=1 (Above average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]*1.2:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]*1.2:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]*1.2:.1f}','|')

print('Sales (T)        | ', f'{value(model.lsb["ABOVE"].W["WHEAT"]):.1f}', '|', 
      f'{value(model.lsb["ABOVE"].W["CORN"]):.1f}', '|',
       f'{value(model.lsb["ABOVE"].W["BEETS_FAVORABLE"]) + value(model.lsb["ABOVE"].W["BEETS_UNFAVORABLE"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.lsb["ABOVE"].Y["WHEAT"]):.1f}', '|', 
      f'{value(model.lsb["ABOVE"].Y["CORN"]):.1f}', '  |',
       '-','     |')

print('First stage: s=2 (Average average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]:.1f}','|')
print('Sales (T)        | ', f'{value(model.lsb["AVERAGE"].W["WHEAT"]):.1f}', '|', 
      f'{value(model.lsb["AVERAGE"].W["CORN"]):.1f}', '  |',
       f'{value(model.lsb["AVERAGE"].W["BEETS_FAVORABLE"]) + value(model.lsb["AVERAGE"].W["BEETS_UNFAVORABLE"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.lsb["AVERAGE"].Y["WHEAT"]):.1f}', '  |', 
      f'{value(model.lsb["AVERAGE"].Y["CORN"]):.1f}', '  |',
       '-','     |')

print('First stage: s=3 (Below average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]*0.8:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]*0.8:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]*0.8:.1f}','|')
print('Sales (T)        | ', f'{value(model.lsb["BELOW"].W["WHEAT"]):.1f}', '|', 
      f'{value(model.lsb["BELOW"].W["CORN"]):.1f}', '  |',
       f'{value(model.lsb["BELOW"].W["BEETS_FAVORABLE"]) + value(model.lsb["BELOW"].W["BEETS_UNFAVORABLE"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.lsb["BELOW"].Y["WHEAT"]):.1f}', '  |', 
      f'{value(model.lsb["BELOW"].Y["CORN"]):.1f}', '  |',
       '-','     |')
print('Overall profit: $',f"{profit_2stage:.1f}")
